In [1]:
import csv
import codecs
import unicodecsv
import requests
from bs4 import BeautifulSoup
import json

all_region_file_name = "./2017Data/2017-Spring-Match-Data-OraclesElixir-2017-03-05.csv"
lpl_match_file_name = "./2017Data/2017LPL_Match.csv"
lpl_player_file_name = "./2017Data/2017LPL_Player.csv"
week_stats_file_name = "./2017Data/Week6Stats.csv"

In [2]:
f = codecs.open(lpl_player_file_name, 'r', 'gb2312', errors='ignore')
lines = f.readlines()
print "All lines number:" + str(len(lines))
lpl_table = []
lpl_define_row = []
for a_idx, a_line in enumerate(lines):
    words = a_line.split(',')
    if a_idx == 0:
        lpl_define_row = words
    if words[0].startswith('2017'):
        lpl_table.append(words)
print "Data lines number:" + str(len(lpl_table))

#for a_idx, a_item in enumerate(lpl_define_row):
    #print a_item, a_idx
#根据周数/小场序号/战队/位置即可获取当前的记录
#例如第一周/第一场/RNG/ADC即可找到lpl_table中的一行

def TranslateNumber(number):
    chinese_numbers = {1:"一", 2:"二", 3:"三", 4:"四", 5:"五", 6:"六", 7:"七", 8:"八", 9:"九"}
    return chinese_numbers[number]

def TranslateWeek(week_number):
    return "第" + TranslateNumber(week_number) + "周"

def TranslateGame(game_number):
    return "第" + TranslateNumber(game_number) + "场"

print TranslateWeek(1)
print TranslateGame(1)

chinese_positions = {"ADC":"ADC", "Jungle":"JUNGLE", "Middle":"MID", "Top":"TOP", "Support":"SUPPORT"}
chinese_teams = {"Edward Gaming":"EDG", "Team WE":"WE",\
                  "Royal Never Give Up":"RNG", "I May":"IM",\
                  "Invictus Gaming":"IG", "OMG":"OMG",\
                  "Newbee":"Newbee", "Vici Gaming":"VG",\
                  "LGD Gaming":"LGD", "Snake Esports":"Snake",\
                  "QG Reapers":"QG", "Game Talents":"GT"}

LPL_DAMAGE_TAKEN_COLUMN = 28
LPL_DAMAGE_TO_CHAMPION_COLUMN = 25
LPL_TOTAL_GOLD_COLUMN = 17
LPL_WEEK_COLUMN = 2
LPL_GAME_COLUMN = 3
LPL_TEAM_COLUMN = 6
LPL_POSITION_COLUMN = 8
LPL_VS_COLUMN = 4

def GetLPLDamageTaken(week_number, game_number, team_name, opposing_team_name, position):
    #print "Get Damage Taken for "
    #print week_number, game_number, team_name, opposing_team_name, position
    week_chinese = TranslateWeek(week_number)
    game_chinese = TranslateGame(game_number)
    position_chinese = chinese_positions[position]
    
    for a_row in lpl_table:
        current_week = a_row[LPL_WEEK_COLUMN].encode('utf8')
        if current_week.startswith(week_chinese):
            current_game = a_row[LPL_GAME_COLUMN].encode('utf8')
            if current_game.startswith(game_chinese):
                target_team = chinese_teams[team_name.strip()]
                current_team = a_row[LPL_TEAM_COLUMN].encode('utf8').strip()
                if current_team == target_team:
                    current_position = a_row[LPL_POSITION_COLUMN].encode('utf8')
                    if current_position.startswith(position_chinese):
                        current_vs = a_row[LPL_VS_COLUMN].encode('utf8').strip()
                        opposing_team = chinese_teams[opposing_team_name.strip()]
                        #print current_vs
                        #print current_team + " vs " + opposing_team
                        if current_vs == current_team + " vs " + opposing_team or current_vs == opposing_team + " vs " + current_team:
                            return (float(a_row[LPL_DAMAGE_TO_CHAMPION_COLUMN]), 
                                    float(a_row[LPL_DAMAGE_TAKEN_COLUMN]),
                                    float(a_row[LPL_TOTAL_GOLD_COLUMN]))
    return (0, 0, 0)

All lines number:931
Data lines number:920
第一周
第一场


In [3]:
all_region_define_row = None
all_region_table = []

PLAYERID_COLUMN = 8

with open(all_region_file_name, 'rb') as csvfile:
    result = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row_idx, row in enumerate(result):
        if row_idx == 0:
            all_region_define_row = row
        else:
            if row[PLAYERID_COLUMN] != "100" and row[PLAYERID_COLUMN] != "200":
                all_region_table.append(row)
        
for a_idx,a_def in enumerate(all_region_define_row):
    print a_def, a_idx

gameid 0
url 1
league 2
split 3
date 4
week 5
game 6
patchno 7
playerid 8
side 9
position 10
player 11
team 12
champion 13
ban1 14
ban2 15
ban3 16
ban4 17
ban5 18
gamelength 19
result 20
k 21
d 22
a 23
teamkills 24
teamdeaths 25
doubles 26
triples 27
quadras 28
pentas 29
fb 30
fbvictim 31
fbtime 32
kpm 33
okpm 34
ckpm 35
fd 36
fdtime 37
teamdragkills 38
oppdragkills 39
elementals 40
oppelementals 41
firedrakes 42
waterdrakes 43
earthdrakes 44
airdrakes 45
elders 46
oppelders 47
herald 48
heraldtime 49
ft 50
fttime 51
firstmidouter 52
firsttothreetowers 53
teamtowerkills 54
opptowerkills 55
fbaron 56
fbarontime 57
teambaronkills 58
oppbaronkills 59
dmgtochamps 60
dmgtochampsperminute 61
dmgshare 62
earnedgoldshare 63
wards 64
wpm 65
wardshare 66
wardkills 67
wcpm 68
visionwards 69
visionwardbuys 70
visiblewardclearrate 71
invisiblewardclearrate 72
totalgold 73
earnedgpm 74
goldspent 75
gspd 76
minionkills 77
monsterkills 78
monsterkillsownjungle 79
monsterkillsenemyjungle 80
cspm 81
css

In [4]:
PLAYERID_COLUMN = 8
DPM_COLUMN = 59 + 2 #每分钟对英雄伤害
GPM_COLUMN = 74 #每分钟金钱
DAMAGE_TO_CHAMPIONS_COLUMN = 58 + 2 #对英雄总伤害
KILL_COLUMN = 19 + 2
DEATH_COLUMN = 20 + 2
ASSIST_COLUMN = 21 + 2
LEAGUE_COLUMN = 2
SPLIT_COLUMN = 3
WEEK_COLUMN = 5 #LPL和其余赛区的格式不同：LPL直接是周数，其他赛区是week.day
GAME_COLUMN = 6
URL_COLUMN = 1 #LPL赛区没有这栏信息
SIDE_COLUMN = 9
POSITION_COLUMN = 10
PLAYER_COLUMN = 11
TEAM_COLUMN = 12
CHAMPION_COLUMN = 13
BAN1_COLUMN = 14
BAN2_COLUMN = 15
BAN3_COLUMN = 16
BAN4_COLUMN = 17
BAN5_COLUMN = 18

GAMELENGTH_COLUMN = 17 + 2
RESULT_COLUMN = 18 + 2#0 for lose, 1 for win
TOTAL_GOLD_COLUMN = 73

CUSTOM_WEEK_COLUMN = 95
CUSTOM_DAMAGE_TAKEN_COLUMN = 96

#rewrite these columns to a new file
with open(week_stats_file_name, 'wb') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    define_row = all_region_define_row
    define_row.append('custom_week')
    define_row.append('custom_damagetaken')
    csvwriter.writerow(define_row)
    for a_idx, a_row in enumerate(all_region_table):
        #fill in a neat week number
        old_week = a_row[WEEK_COLUMN]
        current_league = a_row[LEAGUE_COLUMN]
        new_week = old_week
        if current_league != "LPL":
            week_parts = old_week.split('.')
            new_week = week_parts[0]
        a_row.append(new_week)
        
        #get damage taken from tencent data or from match history url
        damage_taken = 0
        current_game = a_row[GAME_COLUMN]
        current_team = a_row[TEAM_COLUMN]
        current_position = a_row[POSITION_COLUMN]
        opposing_team = None
        if a_idx % 10 < 5:
            opposing_team = all_region_table[a_idx + 5][TEAM_COLUMN]
        else:
            opposing_team = all_region_table[a_idx - 5][TEAM_COLUMN]
        if current_league == "LPL":
            (lpl_damage_to_champion, damage_taken, gold_earned) = GetLPLDamageTaken(int(new_week), int(current_game), current_team, opposing_team, current_position)
            print str(a_idx) + " - LPL damage taken:" + str(damage_taken) + "-" + str(gold_earned)
        else:
            current_url = a_row[URL_COLUMN]
            if current_url == '':
                continue
            if a_idx % 10 == 0:
                url_words = current_url.split('/')
                url_end = url_words[-1].split('&')[0]
                true_url = "https://acs.leagueoflegends.com/v1/stats/game/" + url_words[-2] + "/" + url_end

                r = requests.get(true_url)
                match_detail = json.loads(r.text)
            damage_taken = match_detail['participants'][a_idx % 10]['stats']['totalDamageTaken']
            print str(a_idx) + " - other damage taken:" + str(damage_taken)
        if current_league == "LPL":
            a_row[DAMAGE_TO_CHAMPIONS_COLUMN] = lpl_damage_to_champion
            a_row[TOTAL_GOLD_COLUMN] = gold_earned
        a_row.append(damage_taken)
        
        csvwriter.writerow(a_row)

0 - other damage taken:23349
1 - other damage taken:24929
2 - other damage taken:11564
3 - other damage taken:6576
4 - other damage taken:8530
5 - other damage taken:23223
6 - other damage taken:17764
7 - other damage taken:16451
8 - other damage taken:11360
9 - other damage taken:12502
10 - other damage taken:26106
11 - other damage taken:38005
12 - other damage taken:25428
13 - other damage taken:20050
14 - other damage taken:19009
15 - other damage taken:48433
16 - other damage taken:29124
17 - other damage taken:25298
18 - other damage taken:18808
19 - other damage taken:16329
20 - other damage taken:18474
21 - other damage taken:17680
22 - other damage taken:12893
23 - other damage taken:11098
24 - other damage taken:12393
25 - other damage taken:20439
26 - other damage taken:23788
27 - other damage taken:10915
28 - other damage taken:9612
29 - other damage taken:5649
30 - other damage taken:31594
31 - other damage taken:24163
32 - other damage taken:16537
33 - other damage taken:

ConnectionError: HTTPSConnectionPool(host='acs.leagueoflegends.com', port=443): Max retries exceeded with url: /v1/stats/game/TRLH1/1002100129?gameHash=c8fa6522709aeeca (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x7f0a20288f50>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))

In [6]:
print current_url